In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F

# Importo las clases implementadas en scripts/
from scripts.dataset_torchaudio import SpectrogramTensorDataset
from scripts.train_regression import SmallCNNRegressor

In [ ]:
# GENERAR EL DATASET
    # EJECUTAR scripts/barridoFMconLabels.py para generar los archivos .wav y sus etiquetas
    # Ejecutar scripts/ConversionWavEspec_TorchAudio.py para generar los tensores (scripts/ConversionWavEspec_Librosa.py obsoleto) 

In [3]:
import pandas as pd

# Leer CSV de etiquetas
labels_df = pd.read_csv("Datasets\datasetFMwav\labels.csv")
print(labels_df.head())

# Crear nueva columna con el nombre del tensor 
labels_df["tensor_name"] = labels_df["filename"].str.replace(".wav", ".pt")
print(labels_df.head())


    filename  carrier  ratio  index
0  pru_1.wav      100   0.05    1.5
1  pru_2.wav      100   0.05    2.0
2  pru_3.wav      100   0.05    2.5
3  pru_4.wav      100   0.05    3.0
4  pru_5.wav      100   0.05    3.5
    filename  carrier  ratio  index tensor_name
0  pru_1.wav      100   0.05    1.5    pru_1.pt
1  pru_2.wav      100   0.05    2.0    pru_2.pt
2  pru_3.wav      100   0.05    2.5    pru_3.pt
3  pru_4.wav      100   0.05    3.0    pru_4.pt
4  pru_5.wav      100   0.05    3.5    pru_5.pt


In [4]:
# Usaremos la implementación de SpectrogramTensorDataset ya definida en `scripts/dataset_torchaudio.py`
# Esta carga los .pt y devuelve (tensor, [carrier, ratio, index])

dataset = SpectrogramTensorDataset(
    tensors_dir="Datasets\\datasetFMespec_torchaudio",
    transform=None
)

# Dividir en train/test (80/20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"Dataset total: {len(dataset)}, train: {len(train_dataset)}, test: {len(test_dataset)}")


Dataset total: 15051, train: 12040, test: 3011


In [5]:
# Definir el modelo (usamos la pequeña CNN implementada en scripts/train_regression.py)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    # IMPORTANTE: en un futuro deberíamos intentar usar GPU, depende de la máquina que nos den para ejecutar esto
print('Device:', device)

model = SmallCNNRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

Device: cpu


In [6]:
# Bucle de entrenamiento | De momento aleatorio, deberíamos ver cual es la buena combinación de epochs/batch size/lr
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 100777.7298
Epoch 2, Loss: 75039.0604
Epoch 3, Loss: 67153.3299
Epoch 4, Loss: 59819.8289
Epoch 5, Loss: 54382.8706


In [ ]:
# Evaluación del modelo en el conjunto de test
model.eval()
preds, trues = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        preds.append(outputs.cpu())
        trues.append(labels.cpu())

preds = torch.cat(preds)
trues = torch.cat(trues)

mse = ((preds - trues)**2).mean(0)  # Se calcula MSE (Mean Squared Error) por cada una de las 3 salidas 
print("MSE carrier:", mse[0].item())
print("MSE ratio:", mse[1].item())
print("MSE index:", mse[2].item())

# En mi ejecución de prueba me ha dado:   
# MSE carrier: 138493.484375 
# MSE ratio: 3.073957681655884 
# MSE index: 6.215304851531982
# 
# Estos valores indican que el carrier falla mucho (creo que porque va de 100Hz en 100Hz)              


MSE carrier: 162229.875
MSE ratio: 3.0819621086120605
MSE index: 12.466989517211914


In [ ]:
# Esto tengo entendido que es util pero de momento da igual
#sns.heatmap(cm, annot=True, fmt="d")
#plt.xlabel("Predicted")
#plt.ylabel("True")
#plt.show()


NameError: name 'cm' is not defined

In [9]:
# Se guarda el modelo, no hace falta  
torch.save(model.state_dict(), "cnn_spectrogram.pth")


In [13]:
# Prueba del modelo con un archivo .wav específico
import torchaudio
from scripts.dataset_torchaudio import waveform_to_spectrogram_tensor
from scripts.utils import get_true_labels

# Ruta del archivo .wav a predecir
wav_path = "pru_96.wav"

# Cargar audio
waveform, sample_rate = torchaudio.load(wav_path)

# Convertir a espectrograma (usando la misma lógica que en el dataset)
spec_tensor = waveform_to_spectrogram_tensor(waveform, sample_rate)

# Asegurar dimensiones correctas y añadir batch
spec_tensor = spec_tensor.unsqueeze(0).to(device)

# Obtener valores reales
true_carrier, true_ratio, true_index = get_true_labels(wav_path)

# Mostrar comparación
print("\nValores reales:")
print(f"Carrier: {true_carrier:.2f} Hz")
print(f"Ratio: {true_ratio:.4f}")
print(f"Index: {true_index:.4f}")

# Predecir
model.eval()
with torch.no_grad():
    prediction = model(spec_tensor).cpu().numpy()[0]

# Mostrar resultados
print(f"Predicción:")
print(f"Carrier: {prediction[0]:.2f} Hz")
print(f"Ratio: {prediction[1]:.4f}")
print(f"Index: {prediction[2]:.4f}")



Valores reales:
Carrier: 100.00 Hz
Ratio: 0.3000
Index: 1.5000
Predicción:
Carrier: 123.48 Hz
Ratio: 0.6098
Index: -0.5910
